In [17]:
import numpy as np
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd


In [18]:
from party_code.GraphRec_WWW19_master.Attention import Attention

In [19]:
from Attention import Attention

In [20]:
import sys
import os

In [21]:
sys.path.append(os.path.join('party_code', 'GraphRec_WWW19_master'))

In [22]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict
from UV_Encoders import UV_Encoder
from UV_Aggregators import UV_Aggregator
from Social_Encoders import Social_Encoder
from Social_Aggregators import Social_Aggregator
import torch.nn.functional as F
import torch.utils.data
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os
from time import time

from sklearn.preprocessing import LabelEncoder


In [3]:
"""
## toy dataset 
history_u_lists, history_ur_lists:  user's purchased history (item set in training set), and his/her rating score (dict)
history_v_lists, history_vr_lists:  user set (in training set) who have interacted with the item, and rating score (dict)

train_u, train_v, train_r: training_set (user, item, rating)
test_u, test_v, test_r: testing set (user, item, rating)

# please add the validation set

social_adj_lists: user's connected neighborhoods
ratings_list: rating value from 0.5 to 4.0 (8 opinion embeddings)
"""

"\n## toy dataset \nhistory_u_lists, history_ur_lists:  user's purchased history (item set in training set), and his/her rating score (dict)\nhistory_v_lists, history_vr_lists:  user set (in training set) who have interacted with the item, and rating score (dict)\n\ntrain_u, train_v, train_r: training_set (user, item, rating)\ntest_u, test_v, test_r: testing set (user, item, rating)\n\n# please add the validation set\n\nsocial_adj_lists: user's connected neighborhoods\nratings_list: rating value from 0.5 to 4.0 (8 opinion embeddings)\n"

In [23]:
dir_data = './data/toy_dataset'

path_data = dir_data + ".pickle"
data_file = open(path_data, 'rb')
history_u_lists_, history_ur_lists_, history_v_lists_, history_vr_lists_, train_u_, train_v_, train_r_, test_u_, test_v_, test_r_, social_adj_lists_, ratings_list_ = pickle.load(
    data_file)

In [24]:
mat = loadmat('datasets/rating.mat')  # load mat-file
mdata = mat['rating']  # variable in mat file


In [25]:
mat_net = loadmat('datasets/trustnetwork.mat')  # load mat-file
mnet = mat_net['trustnetwork']  # variable in mat file


In [26]:
mnet = mnet[~((mnet[:,0] == 21354) +  (mnet[:,1] == 21354))]
mnet_un = np.unique(np.r_[mnet[:,0], mnet[:,1]])

In [27]:
fr_mask = np.isin(mdata[:,0], mnet_un)
mdata = mdata[fr_mask]

In [28]:
user_encoder = LabelEncoder()
mdata[:, 0] = user_encoder.fit_transform(mdata[:, 0])
fr1 = user_encoder.transform(mnet[:,0])
fr2 = user_encoder.transform(mnet[:,1])

In [29]:
item_encoder = LabelEncoder()
mdata[:, 1] = item_encoder.fit_transform(mdata[:, 1])

In [30]:
mdata[:,3] -= 1

In [31]:
social_adj_lists = defaultdict(set)
for pair in zip(fr1, fr2):
    social_adj_lists[pair[0]].add(pair[1])
    social_adj_lists[pair[1]].add(pair[0])

len(list(social_adj_lists.keys())) / np.max(user_encoder.classes_)

0.8164305693404313

In [32]:
history_u_lists = defaultdict(list)
history_ur_lists = defaultdict(list)
history_v_lists = defaultdict(list)
history_vr_lists = defaultdict(list)

for i, record in enumerate(mdata):
    user, item, cat, rating = record
    history_u_lists[user].append(item)
    
    history_ur_lists[user].append(rating)
    
    history_v_lists[item].append(user)
    
    history_vr_lists[item].append(rating)
    
    if i%50000 == 0:
        print(i)
    
   

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000


In [33]:
num_rec = mdata.shape[0] 
item_un = np.unique(mdata[:,1], return_index=True)[1]
user_un = np.unique(mdata[:,0], return_index=True)[1]

In [34]:
tr_frac = 0.8
uniq_rec = np.union1d(item_un, user_un)
available_rec = np.setdiff1d(np.arange(num_rec), uniq_rec)
available_rec_tr_num = int(num_rec*(tr_frac - len(uniq_rec)/num_rec))
train_ind = np.random.choice(available_rec, size=available_rec_tr_num, replace=False)
train_ind = np.union1d(train_ind, uniq_rec)

test_ind = ~np.isin(np.arange(num_rec), train_ind)

In [35]:
train_u, train_v, train_r = [], [], []
for record in mdata[train_ind]:
    user, item, cat, rat = record
    
    train_u.append(user)
    train_v.append(item)
    train_r.append(rat)                

In [36]:
test_u, test_v, test_r = [], [], []
for record in mdata[test_ind]:
    user, item, cat, rat = record
    test_u.append(user)
    test_v.append(item)
    test_r.append(rat)        

In [37]:
ratings_list = {i: i-1 for i in range(1,6)}

In [38]:
"""
GraphRec: Graph Neural Networks for Social Recommendation. 
Wenqi Fan, Yao Ma, Qing Li, Yuan He, Eric Zhao, Jiliang Tang, and Dawei Yin. 
In Proceedings of the 28th International Conference on World Wide Web (WWW), 2019. Preprint[https://arxiv.org/abs/1902.07243]
If you use this code, please cite our paper:
```
@inproceedings{fan2019graph,
  title={Graph Neural Networks for Social Recommendation},
  author={Fan, Wenqi and Ma, Yao and Li, Qing and He, Yuan and Zhao, Eric and Tang, Jiliang and Yin, Dawei},
  booktitle={WWW},
  year={2019}
}
```
"""


class GraphRec(nn.Module):

    def __init__(self, enc_u, enc_v_history, r2e):
        super(GraphRec, self).__init__()
        self.enc_u = enc_u
        self.enc_v_history = enc_v_history
        self.embed_dim = enc_u.embed_dim

        self.w_ur1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_ur2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_uv1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_uv2 = nn.Linear(self.embed_dim, 16)
        self.w_uv3 = nn.Linear(16, 1)
        self.r2e = r2e
        self.bn1 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn2 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn3 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn4 = nn.BatchNorm1d(16, momentum=0.5)
        self.criterion = nn.MSELoss()

    def forward(self, nodes_u, nodes_v):
        embeds_u = self.enc_u(nodes_u)
        #print(nodes_v)
        embeds_v = self.enc_v_history(nodes_v)

        x_u = F.relu(self.bn1(self.w_ur1(embeds_u)))
        x_u = F.dropout(x_u, training=self.training)
        x_u = self.w_ur2(x_u)
        x_v = F.relu(self.bn2(self.w_vr1(embeds_v)))
        x_v = F.dropout(x_v, training=self.training)
        x_v = self.w_vr2(x_v)

        x_uv = torch.cat((x_u, x_v), 1)
        x = F.relu(self.bn3(self.w_uv1(x_uv)))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.bn4(self.w_uv2(x)))
        x = F.dropout(x, training=self.training)
        scores = self.w_uv3(x)
        return scores.squeeze()

    def loss(self, nodes_u, nodes_v, labels_list):
        scores = self.forward(nodes_u, nodes_v)
        return self.criterion(scores, labels_list)


def train(model, device, train_loader, test_loader, optimizer, epoch, best_rmse, best_mae):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        batch_nodes_u, batch_nodes_v, labels_list = data
        optimizer.zero_grad()
        loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0:
            rmse, mae = test(graphrec, device, test_loader)
            print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
                epoch, i, running_loss / 100, rmse, mae))
            running_loss = 0.0
    return 0


def test(model, device, test_loader):
    model.eval()
    tmp_pred = []
    target = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in test_loader:
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)
    return expected_rmse, mae

In [ ]:
class Arg():
    def __init__(self):
        pass
args = Arg()   

"""
parser = argparse.ArgumentParser(description='Social Recommendation: GraphRec model')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--embed_dim', type=int, default=64, metavar='N', help='embedding size')
parser.add_argument('--lr', type=float, default=0.001, metavar='LR', help='learning rate')
parser.add_argument('--test_batch_size', type=int, default=1000, metavar='N', help='input batch size for testing')
parser.add_argument('--epochs', type=int, default=100, metavar='N', help='number of epochs to train')
args = parser.parse_args()
"""

args.batch_size = 128
args.embed_dim = 64
args.lr = 0.0005
args.test_batch_size = 256
args.epochs = 6
"""
    'embed_dim': 64,
    'lr' : 0.001,
    'test_batch_size' : 1000,
    'epochs' : 100
}"""

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
use_cuda = False
#if torch.cuda.is_available():
#    use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

embed_dim = args.embed_dim
#dir_data = './data/toy_dataset'

#path_data = dir_data + ".pickle"
#data_file = open(path_data, 'rb')
#history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(
#    data_file)
"""
## toy dataset 
history_u_lists, history_ur_lists:  user's purchased history (item set in training set), and his/her rating score (dict)
history_v_lists, history_vr_lists:  user set (in training set) who have interacted with the item, and rating score (dict)

train_u, train_v, train_r: training_set (user, item, rating)
test_u, test_v, test_r: testing set (user, item, rating)

# please add the validation set

social_adj_lists: user's connected neighborhoods
ratings_list: rating value from 0.5 to 4.0 (8 opinion embeddings)
"""

trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                          torch.FloatTensor(train_r))
testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                         torch.FloatTensor(test_r))
train_loader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=True)
num_users = history_u_lists.__len__()
num_items = history_v_lists.__len__()
num_ratings = ratings_list.__len__()

u2e = nn.Embedding(num_users, embed_dim).to(device)
v2e = nn.Embedding(num_items, embed_dim).to(device)
r2e = nn.Embedding(num_ratings, embed_dim).to(device)

# user feature
# features: item * rating
agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)
enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)
# neighobrs
agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)
enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social,
                       base_model=enc_u_history, cuda=device)

# item feature: user * rating
agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)

# model
graphrec = GraphRec(enc_u, enc_v_history, r2e).to(device)


#optimizer = torch.optim.RMSprop(graphrec.parameters(), lr=args.lr, alpha=0.9)
optimizer = torch.optim.Adam(graphrec.parameters(), lr=args.lr)


best_rmse = np.iinfo(np.int64).max
best_mae = np.iinfo(np.int64).min
endure_count = 0

times_lst = []

for epoch in range(1, args.epochs + 1):
    
    start_time = time()
    train(graphrec, device, train_loader, test_loader, optimizer, epoch, best_rmse, best_mae)
    expected_rmse, mae = test(graphrec, device, test_loader)
    # please add the validation set to tune the hyper-parameters based on your datasets.

    # early stopping (no validation set in toy dataset)
    if best_rmse > expected_rmse:
        best_rmse = expected_rmse
        best_mae = mae
        endure_count = 0
    else:
        endure_count += 1
    
    impl_time = time() - start_time
    print("rmse: %.4f, mae:%.4f " % (expected_rmse, mae))
    print("ep {0} time: ".format(i), impl_time / 60.)
    times_lst.append(impl_time)
    if endure_count > 5:
        break

[1,     0] loss: 0.106, The best rmse/mae: 3.306850 / 3.082040
[1,   100] loss: 5.297, The best rmse/mae: 1.004229 / 0.734379
[1,   200] loss: 0.660, The best rmse/mae: 0.957447 / 0.729506
[1,   300] loss: 0.617, The best rmse/mae: 0.951123 / 0.744298
[1,   400] loss: 0.616, The best rmse/mae: 0.935782 / 0.704204
[1,   500] loss: 0.610, The best rmse/mae: 0.934107 / 0.714491
[1,   600] loss: 0.589, The best rmse/mae: 0.934406 / 0.704859
[1,   700] loss: 0.591, The best rmse/mae: 0.941145 / 0.732113
[1,   800] loss: 0.572, The best rmse/mae: 0.938626 / 0.696559
[1,   900] loss: 0.589, The best rmse/mae: 0.934618 / 0.721407
[1,  1000] loss: 0.605, The best rmse/mae: 0.931418 / 0.709770
[1,  1100] loss: 0.608, The best rmse/mae: 0.929905 / 0.701088
[1,  1200] loss: 0.570, The best rmse/mae: 0.935730 / 0.728892
[1,  1300] loss: 0.590, The best rmse/mae: 0.953522 / 0.759080
[1,  1400] loss: 0.615, The best rmse/mae: 0.942717 / 0.695764
[1,  1500] loss: 0.582, The best rmse/mae: 0.931991 / 0

In [35]:
import gc

In [36]:
gc.collect()

0

In [33]:
from copy import deepcopy

In [34]:
graphrec_1500 = deepcopy(graphrec)

In [32]:
test(graphrec, device, test_loader)

(0.9400082413332418, 0.7444328)

In [31]:
np.iinfo(np.int64).max

9223372036854775807

In [21]:
%load_ext autoreload

In [22]:
%autoreload 2

In [ ]:
%reload_ext Social_Aggregator